In [506]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_add_pool
import torch.nn.functional as F
import networkx as nx
import numpy as np
from torch_geometric.utils.convert import to_networkx, from_networkx
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [507]:
G = nx.read_adjlist('./data/moreno_propro/out.moreno_propro_propro', delimiter=' ', nodetype=int, comments='%')

degree_centrality = nx.degree_centrality(G)

G_networkx = nx.Graph()
G_networkx.add_edges_from(G.edges)

for node in G.nodes:
    G_networkx.nodes[node]['x'] = torch.tensor(node, dtype=torch.float32)
    G_networkx.nodes[node]['y'] = degree_centrality[node]

# Extract node indices and corresponding degree centrality values
x, y = zip(*[(node, degree_centrality[node]) for node in G.nodes])

# Convert x and y to torch tensors
x = torch.tensor(list(x), dtype=torch.float32)
y = torch.tensor(list(y), dtype=torch.float32)

# Reshape x to a column vector
x = x.view(-1, 1)

# Convert x to a NumPy array before using MinMaxScaler
x_array = x.numpy().reshape(-1, 1)

# Use MinMaxScaler to normalize x to the range [-1, 1]
scaler = MinMaxScaler(feature_range=(-1, 1))
x_normalized = scaler.fit_transform(x_array)

# Convert the normalized NumPy array back to a PyTorch tensor
x_normalized = torch.from_numpy(x_normalized).view(-1, 1)

In [508]:
networx_data_torch = from_networkx(G_networkx)

In [509]:
train_data = Data(x=x_normalized, edge_index=networx_data_torch.edge_index, y=y)

In [510]:
train_data

Data(x=[1870, 1], edge_index=[2, 4480], y=[1870])

In [511]:
train_data

Data(x=[1870, 1], edge_index=[2, 4480], y=[1870])

In [512]:
random_integers = np.random.randint(0, len(y), size=int((len(y) / 10)))
random_integers

array([1737,  851, 1177,  364, 1791,   26,  134,  536, 1028,  967,   79,
       1576,  303, 1794, 1336, 1264,  305,  163, 1488,  830,   47, 1645,
       1628,  130,  362, 1719,  977,  625,  466,  995,  743, 1554, 1129,
       1110, 1856, 1144, 1039,  413, 1300,  776,  533, 1226, 1430,  382,
        739,  595,  376,   81, 1145,  318,  715,   11,  581, 1826,  842,
        794,  951, 1373, 1549, 1063,  532, 1627, 1693, 1516,  545, 1213,
        943, 1790,  963, 1193,  541,  488,  179, 1581,  843,  901, 1267,
        369,  802,  958, 1726,  213, 1820, 1722, 1477, 1769, 1298, 1459,
         40, 1475,  652,  490,   56,  234, 1385, 1742,  107,  113, 1103,
       1777,  513,  667, 1705, 1136, 1486, 1227,  195, 1841, 1189,  884,
        874,  502,  225,  849,  143,   24,  488,  637,  812,  679, 1825,
        778, 1854, 1726, 1118, 1850,  960,  783,  729,  519, 1772, 1316,
       1421,  669, 1860, 1611,  702,  648,  573,  235, 1338,  204,  997,
        111, 1529,  757, 1298, 1066,  698,  148,  6

In [513]:
train_mask = torch.full_like(y, True, dtype=bool)
train_mask[random_integers] = False

In [514]:
train_data.train_mask = train_mask

In [515]:
train_data

Data(x=[1870, 1], edge_index=[2, 4480], y=[1870], train_mask=[1870])

In [516]:
class GNN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

model = GNN(in_channels=1, hidden_channels=64, out_channels=1)

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

train_loader = DataLoader([train_data], shuffle=True)

In [517]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    optimizer.zero_grad()
    out = model(train_data.x, train_data.edge_index)
    loss = criterion(out.view(-1)[train_data.train_mask], train_data.y[train_data.train_mask])
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')

print('Training finished.')

Epoch 1/50, Loss: 0.04112575575709343
Epoch 2/50, Loss: 0.0269099622964859
Epoch 3/50, Loss: 0.026830732822418213
Epoch 4/50, Loss: 0.00981150846928358
Epoch 5/50, Loss: 0.019987886771559715
Epoch 6/50, Loss: 0.021723516285419464
Epoch 7/50, Loss: 0.013393673114478588
Epoch 8/50, Loss: 0.007039885502308607
Epoch 9/50, Loss: 0.013396927155554295
Epoch 10/50, Loss: 0.011961124837398529
Epoch 11/50, Loss: 0.0057718465104699135
Epoch 12/50, Loss: 0.010709086433053017
Epoch 13/50, Loss: 0.011011858470737934
Epoch 14/50, Loss: 0.009580664336681366
Epoch 15/50, Loss: 0.011551003903150558
Epoch 16/50, Loss: 0.0056074392050504684
Epoch 17/50, Loss: 0.009979224763810635
Epoch 18/50, Loss: 0.015644537284970284
Epoch 19/50, Loss: 0.011808105744421482
Epoch 20/50, Loss: 0.006336227525025606
Epoch 21/50, Loss: 0.010446101427078247
Epoch 22/50, Loss: 0.01075817085802555


Epoch 23/50, Loss: 0.009850071743130684
Epoch 24/50, Loss: 0.009846709668636322
Epoch 25/50, Loss: 0.008027850650250912
Epoch 26/50, Loss: 0.003512445604428649
Epoch 27/50, Loss: 0.007701144088059664
Epoch 28/50, Loss: 0.011049462482333183
Epoch 29/50, Loss: 0.009606104344129562
Epoch 30/50, Loss: 0.0057497466914355755
Epoch 31/50, Loss: 0.004785141907632351
Epoch 32/50, Loss: 0.007561970967799425
Epoch 33/50, Loss: 0.008266332559287548
Epoch 34/50, Loss: 0.00541770551353693
Epoch 35/50, Loss: 0.004639596212655306
Epoch 36/50, Loss: 0.004956469871103764
Epoch 37/50, Loss: 0.007306922227144241
Epoch 38/50, Loss: 0.00630228454247117
Epoch 39/50, Loss: 0.003182699205353856
Epoch 40/50, Loss: 0.004834024701267481
Epoch 41/50, Loss: 0.0068651167675852776
Epoch 42/50, Loss: 0.007148585747927427
Epoch 43/50, Loss: 0.003720113541930914
Epoch 44/50, Loss: 0.0034609430003911257
Epoch 45/50, Loss: 0.005864565726369619
Epoch 46/50, Loss: 0.004575170110911131
Epoch 47/50, Loss: 0.002205920871347189

In [518]:
# Evaluate the trained model on the test set
model.eval()
with torch.no_grad():
    test_out = model(train_data.x.view(-1, 1), train_data.edge_index)
    test_loss = criterion(test_out.view(-1), train_data.y)
    print(f'Test Loss: {test_loss.item()}')

    

Test Loss: 0.003200588980689645


In [523]:
train_data.y[train_data.train_mask]

tensor([0.0032, 0.0107, 0.0011,  ..., 0.0005, 0.0011, 0.0011])